HELLO EVERYONE!! 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
#sns.get_dataset_names() #if you want to know what we can get

In [5]:
df = sns.load_dataset('titanic')

In [6]:
# df['age'] = df['age'].fillna(df['age'].mean())    # แทนที่ Nan ใน age ด้วย Mean
df.dropna(subset = ['age'], inplace = True)         # Delete Nan Row
#df.dtypes  #check type of Variable

In [7]:
df2 = df.drop(['parch','sibsp','embarked','adult_male','deck','embark_town','alone'], axis = 1) # Del no need row
df2['survived'] = df2.astype('category') # change from Int64 into Category type
df2['pclass'] = df2.astype('object') # change from Int64 into Object type
df2.head()

,survived,pclass,sex,age,fare,class,who,alive
0,0,0,male,22.0,7.2500,Third,man,no
1,1,1,female,38.0,71.2833,First,woman,yes
2,1,1,female,26.0,7.9250,Third,woman,yes
3,1,1,female,35.0,53.1000,First,woman,yes
4,0,0,male,35.0,8.0500,Third,man,no
